In [4]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from torch.nn.functional import cosine_similarity
import fitz  # PyMuPDF
import io
import cv2
import numpy as np
import csv

print("🔧 모델 로딩 중...")
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1]) 
model.eval()
print("✅ 모델 로딩 완료")

# PDF 로드
print("📄 PDF 파일 로드 중...")
doc = fitz.open("test1.pdf")
print(f"✅ PDF 로드 완료 - 총 {len(doc)} 페이지")

# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 로그 파일 설정
output_path = "slide_similarity_log.csv"
csv_file = open(output_path, mode="w", newline="", encoding="utf-8")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["seconds", "page", "similarity"])
print(f"📝 CSV 로그 파일 초기화 완료: {output_path}")

# MSE 유사도 측정 함수
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

# 프레임 → feature 추출
def extract_feature(image):
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image).convert("RGB")
    else:
        image = image.convert("RGB")
    tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        feature = model(tensor).squeeze()
    return feature

# 영상 파일 로딩
print("🎞️ 영상 로딩 중...")
cap = cv2.VideoCapture("test1.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"✅ 영상 로딩 완료 - FPS: {fps:.2f}")
frame_interval = int(fps * 1)

# 상태 변수들
last_slide = None
last_best_page = 0
prev_sim = None
frame_count = 0

# 하이퍼파라미터
mse_threshold = 500
sim_drop_threshold = 0.01
max_search_range = 20

print("🚀 분석 시작")

# 메인 루프
while True:
    ret, frame = cap.read()
    if not ret:
        print("🎬 영상 끝")
        break

    frame_time = frame_count / fps

    if frame_count % frame_interval == 0:
        #print(f"\n🧭 프레임 {frame_count} (시간: {frame_time:.2f}초) 분석 중...")
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray, (320, 240))

        if last_slide is None:
            print("🆕 첫 슬라이드로 설정")
            last_slide = gray
        else:
            diff = mse(gray, last_slide)
            #print(f"🔍 MSE 차이: {diff:.2f}")

            if diff > mse_threshold:
                print("📈 슬라이드 변경 감지됨 → 탐색 시작")
                frame_sim_results = []

                for offset in range(-int(2 * fps), int(2 * fps) + 1, frame_interval):
                    pos = frame_count + offset
                    if pos < 0 or pos >= cap.get(cv2.CAP_PROP_FRAME_COUNT):
                        continue

                    cap.set(cv2.CAP_PROP_POS_FRAMES, pos)
                    ret2, temp_frame = cap.read()
                    if not ret2:
                        continue

                    frame_rgb = cv2.cvtColor(temp_frame, cv2.COLOR_BGR2RGB)
                    feat1 = extract_feature(frame_rgb)

                    # 기본 후보 페이지 설정
                    candidate_range = [-2, -1, 0, 1, 2]
                    candidates = [last_best_page + i for i in candidate_range if 0 <= last_best_page + i < len(doc)]

                    best_page = last_best_page
                    max_sim = -1

                    print(f"🔎 1차 탐색: 후보 페이지 {candidates}")
                    for i in candidates:
                        page = doc[i]
                        pix = page.get_pixmap(dpi=300)
                        img_bytes = pix.tobytes("ppm")
                        feat2 = extract_feature(Image.open(io.BytesIO(img_bytes)))
                        sim = cosine_similarity(feat1.unsqueeze(0), feat2.unsqueeze(0)).item()

                        print(f"    페이지 {i} 유사도: {sim:.4f}")
                        if sim > max_sim:
                            max_sim = sim
                            best_page = i
                            best_feat2 = feat2

                    # 유사도 급락 시 재탐색
                    if prev_sim is not None and (prev_sim - max_sim) >= sim_drop_threshold:
                        print(f"⚠️ 유사도 하락 감지 (이전: {prev_sim:.4f} → 현재: {max_sim:.4f}) → 2차 탐색")
                        expanded_range = list(range(-(max_search_range//2), max_search_range//2 + 1))
                        expanded_candidates = [last_best_page + i for i in expanded_range if 0 <= last_best_page + i < len(doc)]

                        print(f"🔍 2차 탐색: 후보 페이지 {expanded_candidates}")
                        for i in expanded_candidates:
                            page = doc[i]
                            pix = page.get_pixmap(dpi=300)
                            img_bytes = pix.tobytes("ppm")
                            feat2 = extract_feature(Image.open(io.BytesIO(img_bytes)))
                            sim = cosine_similarity(feat1.unsqueeze(0), feat2.unsqueeze(0)).item()

                            print(f"    [2차] 페이지 {i} 유사도: {sim:.4f}")
                            if sim > max_sim:
                                max_sim = sim
                                best_page = i
                                best_feat2 = feat2

                    result_time = pos / fps
                    minutes = int(result_time // 60)
                    seconds = int(result_time % 60)
                    print(f"✅ [결과] {minutes}분 {seconds}초: 페이지 {best_page} / 유사도 {max_sim:.4f}")
                    csv_writer.writerow([round(result_time, 2), best_page, round(max_sim, 4)])
                    frame_sim_results.append((max_sim, best_page, result_time, gray))

                if frame_sim_results:
                    best_result = max(frame_sim_results, key=lambda x: x[0])
                    prev_sim = best_result[0]
                    last_best_page = best_result[1]
                    last_slide = best_result[3]
                    print(f"📌 현재 상태 갱신 → 페이지 {last_best_page}, 유사도 {prev_sim:.4f}")

    frame_count += 1

cap.release()
csv_file.close()
print("✅ 모든 작업 완료. 로그 저장됨.")


🔧 모델 로딩 중...


c:\Users\good1\anaconda3\envs\neuralforecast_real\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\good1\anaconda3\envs\neuralforecast_real\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ 모델 로딩 완료
📄 PDF 파일 로드 중...
✅ PDF 로드 완료 - 총 33 페이지
📝 CSV 로그 파일 초기화 완료: slide_similarity_log.csv
🎞️ 영상 로딩 중...
✅ 영상 로딩 완료 - FPS: 30.00
🚀 분석 시작
🆕 첫 슬라이드로 설정
📈 슬라이드 변경 감지됨 → 탐색 시작
🔎 1차 탐색: 후보 페이지 [0, 1, 2]
    페이지 0 유사도: 0.9955
    페이지 1 유사도: 0.8803
    페이지 2 유사도: 0.7426
✅ [결과] 0분 29초: 페이지 0 / 유사도 0.9955
🔎 1차 탐색: 후보 페이지 [0, 1, 2]
    페이지 0 유사도: 0.9955
    페이지 1 유사도: 0.8803
    페이지 2 유사도: 0.7426
✅ [결과] 0분 30초: 페이지 0 / 유사도 0.9955
🔎 1차 탐색: 후보 페이지 [0, 1, 2]
    페이지 0 유사도: 0.8840
    페이지 1 유사도: 0.9950
    페이지 2 유사도: 0.8551
✅ [결과] 0분 31초: 페이지 1 / 유사도 0.9950
🔎 1차 탐색: 후보 페이지 [0, 1, 2]
    페이지 0 유사도: 0.8840
    페이지 1 유사도: 0.9950
    페이지 2 유사도: 0.8551
✅ [결과] 0분 32초: 페이지 1 / 유사도 0.9950
🔎 1차 탐색: 후보 페이지 [0, 1, 2]
    페이지 0 유사도: 0.8840
    페이지 1 유사도: 0.9950
    페이지 2 유사도: 0.8551
✅ [결과] 0분 33초: 페이지 1 / 유사도 0.9950
📌 현재 상태 갱신 → 페이지 0, 유사도 0.9955
📈 슬라이드 변경 감지됨 → 탐색 시작
🔎 1차 탐색: 후보 페이지 [0, 1, 2]
    페이지 0 유사도: 0.8836
    페이지 1 유사도: 0.9941
    페이지 2 유사도: 0.8560
✅ [결과] 2분 53초: 페이지 1 / 유사도 0.9941
🔎 1차 탐색: 후보 페이

In [1]:
# PDF 파일에서 페이지별 텍스트 추출 및 CSV로 저장
import fitz  # PyMuPDF
import pandas as pd

# PDF 파일 열기
pdf_path = 'test1.pdf'
doc = fitz.open(pdf_path)

# 페이지별 텍스트 저장
data = []
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    text = page.get_text()
    data.append({
        'page': page_num,
        'text': text.strip()
    })

# DataFrame으로 저장
df = pd.DataFrame(data)

# CSV 파일로 저장
csv_path = 'test1_text_by_page.csv'
df.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'CSV로 저장 완료: {csv_path}')

CSV로 저장 완료: test1_text_by_page.csv


In [2]:
# 텍스트 파일에서 필요한 정보 추출
import re
import pandas as pd

parsed = []

with open(r"C:\Users\good1\Desktop\summer_vacation\smartnote\gitproject\whisper\transcription_large_timestamp.txt", "r", encoding="utf-8") as f:
    for line in f:
        # 한 줄에 여러 [시간 --> 시간] 블록이 있을 수 있음
        matches = re.findall(r"\[(\d{2}:\d{2}:\d{2}\.\d{3}|\d{2}:\d{2}\.\d{3}) --> (\d{2}:\d{2}:\d{2}\.\d{3}|\d{2}:\d{2}\.\d{3})\]\s*([^[]+)", line)
        for match in matches:
            start, end, text = match
            parsed.append({
                "start": start,
                "end": end,
                "text": text.strip()
            })

df_transcript = pd.DataFrame(parsed)

def time_to_seconds(tstr):
    """시간 문자열을 초(float)로 변환"""
    parts = tstr.split(":")
    if len(parts) == 3:  # hh:mm:ss.ms
        h, m, s = parts
    else:  # mm:ss.ms
        h = 0
        m, s = parts
    s, ms = s.split(".")
    total_seconds = int(h) * 3600 + int(m) * 60 + int(s) + int(ms) / 1000
    return total_seconds
match_df = pd.read_csv("slide_similarity_log2.csv")  # 또는 위 테이블을 직접 DataFrame으로 만듦
df_transcript["end_sec"] = df_transcript["end"].apply(time_to_seconds)

def get_page(sec):
    # 해당 시간보다 작거나 같은 seconds 중 가장 큰 것의 page 찾기
    candidates = match_df[match_df["seconds"] <= sec]
    if not candidates.empty:
        return int(candidates.iloc[-1]["page"])
    return 0 

df_transcript["page"] = df_transcript["end_sec"].apply(get_page)
df_transcript.to_csv("transcript_with_page.csv", index=False, encoding='utf-8-sig')

In [1]:
import pandas as pd


df = pd.read_csv('transcript_with_page.csv')

grouped = df.groupby('page', as_index=False).agg({
    'text': ' '.join  
})

print(grouped)

#grouped.to_csv('merged_text_by_page.csv', index=False)

    page                                               text
0      0  꼼꼼한 딥러닝 논문 리뷰와 코드 실습. 이번 시간에 리뷰할 논문은 현대 딥러닝 기반...
1      1  실제로 트랜스포머는 Attention이라는 메커니즘을 전적으로 활용하는 아키텍처입니...
2      2  그렇다면 기존에 제한되었던 시퀀스 투 시퀀스 모델에는 어떤 한계점이 존재할까요? 기...
3      3  그래서 이러한 문제를 조금이나마 완화하기 위한 아이디어로, 이 고정된 크기의 컨텍스...
4      4  즉 현재의 문제 상황이라고 한다면, 하나의 문맥 벡터, 즉 컨텍스트 벡터가, 소스 ...
5      5  지금 보이는 아키텍처가 바로, 시퀀스 시퀀스에 어텐션 메커니즘을 적용한 아키텍처인데...
6      6  자 그래서 실제로, 이 attention 기법을 적용했을 때, 디코더 파트에서 각각...
7      7  그래서 이 그림은 실제로, 이 attention mechanism을 제안한 논문에서...
8      8  이제 이런 식으로 attention 메커니즘을 쓰게 되면, 성능이 좋아질 뿐만 아니...
9      9  그렇다면 오늘 리뷰하고 있는 트랜스포머 논문은, 어떤 원리로 동작할까요? 트랜스포머...
10    10  오른쪽에 보이는 그림이, 원본 논문에서 보여주고 있는, 트랜스포머의 아키텍처인데요....
11    11  어떠한 단어 정보를, 네트워크에 넣기 위해서는 일반적으로, 보통 인베딩 과정을 거칩...
12    12  이때 우리가, 시퀀스 투 시퀀스와 같은, RNN 기반의, 아키텍처를 사용한다고 하면...
13    13  위치에 대한 정보까지 포함하고 있는 입력 값을, 실제 어텐션에 넣어줄 수 있도록 합...
14    14  또한 여기에서 추가적으로, residual learning과 같은 테크닉이 사용되는...
15    15  그래서 실제로 이렇게 입력 값이 들어온 이후로부터, a

In [ ]:
import pandas as pd
import requests

# CSV 파일 읽기
example_df = pd.read_csv('transcript_with_page.csv')
#example_df = grouped
page_df = pd.read_csv('test1_text_by_page.csv')

# Ollama 서버 URL
OLLAMA_URL = 'http://localhost:11434/api/generate'

# 결과 저장용 리스트
results = []

# 예시로 example2.csv의 각 row마다 실행
for idx, row in example_df.iterrows():
    text1 = row['text']  # example2.csv에는 'text' 컬럼이 있다고 가정

    page_num = row['page']  # page 번호
    # page_df에서 해당하는 page의 text 가져오기
    matched_rows = page_df[page_df['page'] == page_num]

    if matched_rows.empty:
        print(f"❗ page {page_num}에 해당하는 텍스트가 없습니다.")
        continue

    text2 = matched_rows.iloc[0]['text']

    # 프롬프트 구성
    prompt = f"""
    문장1의 내용을 바탕으로 문장2를 보완해줘. 단, 문장1에 없는 내용은 추가하지 말고 문장2 안의 표현이나 맥락을 더 구체화하거나 풍부하게 해줘.

    결과는 보완된 문장2만 출력해. 시스템 프롬프트나 설명은 생략하고, 너의 사고과정도 드러내지 마.

    문장1: {text1}
    문장2: {text2}
    """

    # 요청 payload
    payload = {
        "model": "gemma3:12b",
        "prompt": prompt,
        "stream": False
    }

    # 요청 보내기
    response = requests.post(OLLAMA_URL, json=payload)

    # 응답 처리
    if response.status_code == 200:
        result = response.json()
        print(f"(index {idx}):")
        print(result['response'])
        results.append({
            'index': idx,
            'page': page_num,
            'text1': text1,
            'text2': text2,
            'gemma_response': result['response']
        })
    else:
        print(f"❌ 요청 실패 (index {idx}): {response.status_code}")
        print(response.text)

# 필요하면 결과를 CSV로 저장
output_df = pd.DataFrame(results)
output_df.to_csv('gemma_opinions.csv', index=False)


(index 0):
자연어처리: 트랜스포머
Transformer (Attention Is All You Need) - 딥러닝 기반 자연어처리 기술의 핵심 아키텍처로 자리 잡은 트랜스포머 모델에 대한 심층적인 이해를 돕기 위한 리뷰입니다.
나동빈(dongbinna@postech.ac.kr)
Pohang University of Science and Technology
(index 1):
Transformer (Attention Is All You Need)는 2017년 발표된 획기적인 자연어 처리 모델로, "Attention is all you need"라는 제목에서 명확히 드러나듯, Attention 메커니즘을 핵심 아이디어로 활용하여 기존 순환 신경망(RNN)의 단점을 극복하고 뛰어난 성능을 달성했다.
(index 2):
2021년 기준으로 최신 고성능 모델들은 Transformer 아키텍처를 기반으로 하며, 이는 Attention 메커니즘을 핵심으로 활용하는 구조입니다. GPT는 Transformer의 디코더(Decoder) 아키텍처를 활용하여 텍스트 생성 능력을 극대화하고, BERT는 Transformer의 인코더(Encoder) 아키텍처를 활용하여 문맥 이해 능력을 향상시켰습니다. 이러한 발전은 1986년 RNN(Recurrent Neural Network)의 등장으로 시작되었고, 1997년 LSTM(Long Short-Term Memory)이 RNN의 단점을 보완하며 기계 번역 분야의 발전을 이끌었습니다. 이후 Seq2Seq 모델(NIPS 2014)이 등장하며 입력 시퀀스 전체에서 정보를 추출하는 방향으로 발전했고, 2015년 Attention 메커니즘이 등장하며 문장 내 중요한 부분에 집중하는 능력이 더해졌습니다. 이러한 Attention 메커니즘을 전적으로 활용한 Transformer 아키텍처(NIPS 2017)는 GPT-1(2018), BERT(NAACL 2019), GPT-3(2020)와 같은 획기적인 모델의 기반이 되었습니다.
(index 3

In [2]:
import pandas as pd
import requests

# CSV 파일 읽기
#example_df = pd.read_csv('example2.csv')
example_df = grouped
page_df = pd.read_csv('test1_text_by_page.csv')

# Ollama 서버 URL
OLLAMA_URL = 'http://localhost:11434/api/generate'

# 결과 저장용 리스트
results = []

# 예시로 example2.csv의 각 row마다 실행
for idx, row in example_df.iterrows():
    text1 = row['text']  # example2.csv에는 'text' 컬럼이 있다고 가정

    page_num = row['page']  # page 번호
    # page_df에서 해당하는 page의 text 가져오기
    matched_rows = page_df[page_df['page'] == page_num]

    if matched_rows.empty:
        print(f"❗ page {page_num}에 해당하는 텍스트가 없습니다.")
        continue

    text2 = matched_rows.iloc[0]['text']

    # 프롬프트 구성
    prompt = f"""
    문장1의 내용을 바탕으로 문장2를 보완해줘. 단, 문장1에 없는 내용은 추가하지 말고 문장2 안의 표현이나 맥락을 더 구체화하거나 풍부하게 해줘.

    결과는 보완된 문장2만 출력해. 시스템 프롬프트나 설명은 생략하고, 너의 사고과정도 드러내지 마.

    문장1: {text1}
    문장2: {text2}
    """

    # 요청 payload
    payload = {
        "model": "gemma3:12b",
        "prompt": prompt,
        "stream": False
    }

    # 요청 보내기
    response = requests.post(OLLAMA_URL, json=payload)

    # 응답 처리
    if response.status_code == 200:
        result = response.json()
        print(f"(index {idx}):")
        print(result['response'])
        results.append({
            'index': idx,
            'page': page_num,
            'text1': text1,
            'text2': text2,
            'gemma_response': result['response']
        })
    else:
        print(f"❌ 요청 실패 (index {idx}): {response.status_code}")
        print(response.text)

# 필요하면 결과를 CSV로 저장
output_df = pd.DataFrame(results)
output_df.to_csv('gemma_opinions.csv', index=False)


(index 0):
자연어처리: 트랜스포머
Transformer (Attention Is All You Need)
나동빈(dongbinna@postech.ac.kr)
Pohang University of Science and Technology
본 자료는 현대 딥러닝 기반 자연어처리 기술의 핵심 아키텍처로 자리 잡은 트랜스포머 모델, 특히 "Attention is all you need"라는 제목의 획기적인 논문을 심층적으로 이해하기 위한 것입니다.
(index 1):
딥러닝 기반의 기계 번역 발전 과정을 살펴보면, 1986년 RNN의 등장 이후 약 10년 뒤 LSTM이 등장하여 다양한 시퀀스 정보 모델링을 가능하게 했고, 2014년에는 LSTM을 활용한 시퀀스 투 시퀀스 모델이 제안되어 현대 딥러닝 기술 발전의 중요한 전환점이 되었습니다. 하지만 이 모델은 소스 문장을 고정된 크기의 컨텍스트 벡터로 압축해야 한다는 한계에 직면했고, 이후 어텐션 메커니즘의 도입으로 이러한 제약이 완화되었습니다. 2017년 NIPS에서 발표된 Transformer 논문은 어텐션 메커니즘 자체를 핵심으로 하는 아키텍처를 제시하며 성능을 획기적으로 향상시켰고, 2021년 기준으로 최신 고성능 모델들은 Transformer 아키텍처를 기반으로 하고 있습니다. 특히 GPT는 Transformer의 디코더 아키텍처, BERT는 인코더 아키텍처를 활용하여 뛰어난 성능을 보여주고 있으며, GPT-3(2020)와 같은 후속 모델들은 이러한 트렌드를 이어가고 있습니다. 이러한 발전은 입력 시퀀스 전체에서 정보를 추출하는 방향으로 연구가 발전하는 계기가 되었으며, 어텐션 메커니즘을 더욱 효과적으로 활용하는 다양한 고성능 모델들이 등장하는 추세로 이어지고 있습니다.
(index 2):
기존 Seq2Seq 모델들은 소스 문장의 정보를 단일하고 고정된 크기의 컨텍스트 벡터 𝑣로 압축하는 방식으로 동작합니다. 이는 입력 문장의 길이가 다양함에도 불구하고, 모든 정보를 제한된 크기의 벡터에 담아내야 한다